## Setup MLFlow to be used in the experiment

In [7]:
import mlflow
from lib.constants import MLFLOW_URI

EXPERIMENT_NAME = 'sample-sklearn'
mlflow.set_tracking_uri(uri=MLFLOW_URI)
experiment = mlflow.set_experiment(EXPERIMENT_NAME)

## Log MLFlow model run

In [34]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define model hyperparameters
params = {
    'solver': 'lbfgs',
    'max_iter': 1000,
    'multi_class': 'auto',
    'random_state': 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Log parameters, metrics, and the model
with mlflow.start_run() as run:
    mlflow.log_params(params)

    accuracy = lr.score(X_test, y_test)
    mlflow.log_metric('accuracy', accuracy)

    mlflow.sklearn.log_model(lr, 'model')

    last_run_id = run.info.run_id

## List model runs

In [ ]:
model_runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])
model_runs[[
    'run_id',
    'status',
    'metrics.accuracy'
]]

## Load model run

In [ ]:
best_run_id = (
    model_runs
    .assign(total_time=lambda df: df.apply(lambda row: (row.end_time - row.start_time).total_seconds(), axis=1))
    .sort_values(['metrics.accuracy', 'total_time'], ascending=[False, True])
    .iloc[0]
    .run_id
)

loaded_model = mlflow.sklearn.load_model(f'runs:/{last_run_id}/model')
accuracy = loaded_model.score(X_test, y_test)
predictions = loaded_model.predict(X_test)
print('Accuracy from inferece model to test data:', accuracy)